In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from nbdev import *
%nbdev_default_export core
%nbdev_default_class_level 3

Cells will be exported to geomechy.core,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
import numpy as np
from geomechy.base import ItemDict
from geomechy.utils import *
from geomechy.io import jsonReader
from geomechy.materials import *
from geomechy.shape import *

# Core
> Main program

## Node Set Object

In [ ]:
%nbdev_export
class NodeSet(ItemDict):
        
    def readFromDict(self, data):
        nodes_dict = data["Nodes"]
        self.node_dim = eval(nodes_dict["dim"])
        
        for node in nodes_dict["coords"]:
            self.add(node[0], np.array(node[1:])*self.node_dim)
    
    def getNodeCoords(self, nodeId):
        return self.get(nodeId)

## Element Set Object

In [ ]:
%nbdev_export
class ElementSet(ItemDict):
        
    def readFromDict(self, data):
        elems_dict = data["Elements"]
        self.elementType = elems_dict["elementType"]
        
        for elem in elems_dict["elems"]:
            self.add(elem[0], elem[1:])
    
    def getElementNodes(self, elemId):
        return self.get(elemId)

## Material Set Object

In [ ]:
%nbdev_export
class MaterialSet(dict):
    
    def __init__(self, material):
        
        fluids = material["Fluids"]
        media  = material["PorousMedia"]
        temp   = material["Temp"]
        
        for idx, f in enumerate(fluids):
            if f["Type"] == "Water":
                w = Water(f)
                self[f"water_{idx+1}"] = w
            elif f["Type"] == "Oil":
                o = Oil(f)
                self[f"oil_{idx+1}"] = o
            elif f["Type"] == "Gas":
                g = Gas(f)
                self[f"gas_{idx+1}"] = g
            elif f["Type"] == "Air":
                a = Air(f)
                self[f"air_{idx+1}"] = a
            else:
                pass
                
        for idx, m in enumerate(media):
            if m["Type"] == "Soil":
                s = Soil(m)
                self[f"soil_{idx+1}"] = s
            elif m["Type"] == "Rock":
                r = Rock(m)
                self[f"rock_{idx+1}"] = r
            else:
                pass  

## Shape Functions Manager Object

In [ ]:
%nbdev_export
class ShapeFunctionsManager(ItemDict):
    
    def __init__(self, nodeSet, elemSet, reduced=False):
        self.nodeSet = nodeSet
        self.elemSet = elemSet
        self.reduced = reduced
        
        if not self.elemSet.elementType:
            e1_nodes  = self.elemSet.getElementNodes(1)
            e1_coords = np.array([self.nodeSet.getNodeCoords(c) for c in e1_nodes])
            self.elementType = getElemetType(e1_coords)
        else:
            self.elementType = self.elemSet.elementType
            
        self.gp, self.we = getGaussPoints(elemType=self.elementType, reduced=self.reduced)          
    
    def getShapeFunc(self):
        all_N = []
        all_dN = []
        for gp in self.gp:
            N, dN = getAllShapeFunctions(self.elementType, gp)
            all_N.append(N)
            all_dN.append(dN)
        self.N, self.dN = all_N, all_dN
        
        for key, value in self.elemSet.items():
            self[key] = {"gp": self.gp, "we": self.we,"N":self.N, "dN":self.dN}
            

## Constitutive Manager

In [ ]:
%nbdev_export
class ConstitutiveManager(list):
    
    def __init__(self, matProps):
        matType = matProps.type
        exec(f'from geomechy.materials import {matType} as material', globals())
        self.mat = material(matProps)
        self.iIter = -1

    def reset(self):
        self.iIter = -1

    def getStress(self, kinematic, iSam=-1):

        if iSam == -1:
            self.iIter += 1
            iSam = self.iIter

        self.mat.setIter(iSam)
        return self.mat.getStress(kinematic)

    def commitHistory(self):
        self.mat.commitHistory()

## Element Maneger Object

In [ ]:
%nbdev_export
class ElementManager(ItemDict):
    
    def __init__(self, nodes, props):
        ItemDict.__init__(self)
        self.nodes  = nodes
        self.props  = props
        self.groups = {}
    
    def __iter__(self):
        elements = []

        for groupName in self.iterGroupNames():
            for element in self.iterElementGroup(groupName):
                elements.append(element)

        return iter(elements)
    
    
    def getDofTypes(self):
        dofTypes = []
        
        for element in self:
            for dofType in element.dofTypes:
                if dofType not in dofTypes:
                    dofTypes.append(dofType)
        return dofTypes
    
    
    def readFromFile(self, fileName):
        fin = open(filename)
        
    
    def add(self, ID, modelName, elementNodes):  

        #Check if the model exists
        if not hasattr(self.props, modelName):
            RuntimeError('Missing properties for model ' + modelName)

        modelProps = getattr(self.props, modelName)

        
        #Check if the model has a type
        if not hasattr( modelProps, 'type' ):
            RuntimeError('Missing type for model ' + modelName)

        modelType = getattr(modelProps, 'type')

        
        #Load the element
        exec(f'from geomechy.elements import {modelType} as element',globals())

        
        #Create the element
        elem = element(elementNodes, modelProps)

        #Check if the node IDs are valid
        for nodeID in elem.getNodes():
            if not nodeID in self.nodes:
                raise RuntimeError('Node ID ' + str(nodeID) + ' does not exist')

        
        #Add the element to the element set
        ItemDict.add(self, ID, elem)

        #Add the element to the correct group
        self.addToGroup(modelName, ID)
    
    
    def addToGroup(self, modelType, ID):

        if not modelType in self.groups:
            self.groups[modelType] = [ID]
        else:
            self.groups[modelType].append( ID )

    def iterGroupNames(self):
        return self.groups
        
    def iterElementGroup (self, groupName):
        if groupName == "All":
            return iter(self)
        else:
            return iter(self.get(self.groups[groupName]))

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_physics.ipynb.
Converted 02_constitutive.ipynb.
Converted 03_shape.ipynb.
Converted 04_io.ipynb.
Converted 06_materials.ipynb.
Converted 08_Modules.ipynb.
Converted 09_base.ipynb.
Converted 10_utils.ipynb.
Converted index.ipynb.
Converted test.ipynb.
